In [1]:
# install.packages(c("readxl", "readr", "scales", "lavaan"))

In [2]:
library(dplyr)
library(tidyr)
library(stringr)
library(readxl)    # For reading Excel files
library(readr)     # For reading CSV files
library(lavaan)
library(jsonlite)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.



# Prepare phenotype

In [3]:
pheno_df <- read_csv("/mnt/project/notebooks/bmi/data/pheno.csv.gz", col_types = cols(sample_names = col_character()))

In [4]:
pheno_df <- pheno_df[pheno_df$ancestry_pred != "oth", ]

In [5]:
dim(pheno_df)

[1] 454645     44

In [6]:
# Function for rank-based inverse normal transformation
rint_normalization <- function(ser) {
  ranks <- rank(ser, na.last = "keep")  # Compute ranks
  normalized <- qnorm((ranks - 0.5) / sum(!is.na(ranks)))  # Apply inverse normal transformation
  return(normalized)
}

# Function to normalize covariates
normalize_covariates <- function(pheno_df, covariates, exclude = character()) {
  norm_pheno_df <- pheno_df
  for (cov in covariates) {
    if (!(cov %in% exclude)) {
      norm_pheno_df[[cov]] <- scale(norm_pheno_df[[cov]])  # Standardize column
    }
  }
  return(norm_pheno_df)
}

# Apply transformations
pheno_df <- pheno_df %>%
  group_by(ancestry_pred, genetic_sex) %>%
  mutate(bmi_rint = rint_normalization(bmi)) %>%  # RINT for BMI
  ungroup() %>%
  mutate(
    genetic_sex = as.integer(genetic_sex == "Female"),  # Convert genetic_sex to binary
    age_2 = age^2,                                    # Add quadratic age
    age_sex = age * genetic_sex                       # Interaction term
  )



# Normalize specified covariates
covariates <- c("age", "age_2", "age_sex", "genetic_sex", paste0("genetic_pca", 1:10)) # "bmi_prs", 
norm_pheno_df <- normalize_covariates(pheno_df, covariates, exclude = c("genetic_sex"))

# Build the regression model
equation <- paste("bmi_rint ~", paste(covariates, collapse = " + "))
formula <- as.formula(equation)

# Fit the model using base R
model <- lm(formula, data = norm_pheno_df)

# Extract residuals
norm_pheno_df$bmi_residuals <- residuals(model)

# Create gene burden

In [7]:
# Function to create a gene burden table helper
create_gene_burden_table_helper <- function(burden_df, annotations, maf, lf_samples_df, hgnc_dict) {
  # Map the 'gene' column using hgnc_dict
  burden_df$gene <- hgnc_dict[burden_df$gene]
  
  # Subset the burden_df based on annotations and maf_max
  masked_burden_df <- burden_df[
    burden_df$annotation %in% annotations & burden_df$maf_max <= maf, 
    c("gene", "samples")
  ]
  
    
  masked_burden_df <- masked_burden_df %>%
    group_by(gene) %>%
    summarise(samples = list(unique(unlist(strsplit(paste(samples, collapse=","), split=",")))))

  
  # Combine with lf_samples_df (assumed to be a data frame with gene and samples columns)
  masked_burden_df <- rbind(masked_burden_df, lf_samples_df)
  
  return(masked_burden_df)
}

# Function to create gene burden tables
create_gene_burden_tables <- function(burden_df, maf, lf_samples_df, hgnc_dict) {
  masks <- c("pLoF", "Missense_strict", "Missense_lenient")
  annot_terms <- list(c("lof"), 
                      c("lof", "missense_strict"), 
                      c("lof", "missense_strict", "missense_lenient"))
  
  # Create a list of gene burden tables for each annotation term
  gene_burden_dict <- setNames(
    lapply(annot_terms, function(at) create_gene_burden_table_helper(burden_df, at, maf, lf_samples_df, hgnc_dict)),
    masks
  )
  
  return(gene_burden_dict)
}

      
get_samples_helper <- function(combos, genotype_df, cohort_samples) {
  if (length(intersect(combos, genotype_df$gene)) == length(combos)) {
    samples_per_gene <- genotype_df %>%
      filter(gene %in% combos) %>%
      pull(samples)

    samples_per_combo <- Reduce(intersect, samples_per_gene)
    samples_per_combo <- intersect(cohort_samples, samples_per_combo)
  } else {
    samples_per_combo <- character(0)
  }
  
  return(samples_per_combo)
}

                                        
get_samples <- function(ser, gene_burden_dict, pop_samples) {
  # Extract gene and mask from the input `ser`
  gene <- ser$gene
  mask <- ser$gene_mask
  
  # Access the gene samples dataframe from `gene_burden_dict` using the mask
  gene_samples_df <- gene_burden_dict[[mask]]
  
  # Initialize combos with the gene
  combos <- c(gene)
  
  # Check if "lf" exists in `ser` and add it to combos if present
  if ("lf" %in% names(ser)) {
    lf <- ser$lf
    combos <- c(combos, lf)
  }
  
  # Call the helper function with combos, gene_samples_df, and pop_samples
  samples <- get_samples_helper(combos, gene_samples_df, pop_samples)
  
  return(samples)
}

In [8]:
# Load gene burden data
gene_burden_df <- read.delim(
  file = "/mnt/project/notebooks/wes/burden_preparation/data/ukb_burden.tsv.gz",
  sep = "\t",
  header = TRUE
)

# Load gnomAD annotation data
gnomad_df <- read.delim(
  file = "/mnt/project/notebooks/wes/burden_preparation/data/gnomad_annot.tsv.gz",
  sep = "\t",
  header = TRUE,
  colClasses = c("locus" = "character", "alleles" = "character", "maf_gnomad_popmax" = "numeric")
)[, c("locus", "alleles", "maf_gnomad_popmax")]

# Merge dataframes on 'locus' and 'alleles'
gene_burden_df <- merge(gene_burden_df, gnomad_df, by = c("locus", "alleles"))

# Calculate the maximum MAF
gene_burden_df$maf_max <- apply(gene_burden_df[, c("maf", "maf_gnomad_popmax")], 1, max)


In [9]:
# Read the JSON file into an R list
hgnc_dict <- fromJSON("/mnt/project/notebooks/bmi/data/hgnc_gene_map.json")

In [10]:
# Assuming the function 'create_gene_burden_tables' is already implemented in R

# Create gene burden tables
gene_burden_dict <- create_gene_burden_tables(gene_burden_df, 0.001, data.frame(), hgnc_dict)

# Convert sample names to a set (unique values)
pop_samples <- unique(as.character(pheno_df$sample_names))


# Conduct SEM

In [11]:
create_gene_pheno_df <- function(ser, gene_burden_dict, pop_samples, pheno_df) {
  # Extract gene, lifestyle, mask, and sample information
    gene_samples <- get_samples(ser, gene_burden_dict, pop_samples)

    # Create a copy of the phenotype data frame
    gene_pheno_df <- pheno_df

    # Add columns for gene, lifestyle factor, and gene-lifestyle factor presence
    gene_pheno_df$gene <- as.integer(gene_pheno_df$sample_names %in% gene_samples)
    comorbidity <- ser$comorbidity
    gene_pheno_df$comorbidity <- as.integer(gene_pheno_df[[comorbidity]])
    # Select relevant columns and drop rows with NA values
    gene_pheno_df <- gene_pheno_df %>%
    select(bmi_residuals, comorbidity, gene) %>%
    drop_na()
  
  return(gene_pheno_df)
}


run_sem_model <- function(ser, gene_burden_dict, pop_samples, pheno_df){
    gene <- ser$gene
    comorbidity <- ser$comorbidity
    
    df <- create_gene_pheno_df(ser, gene_burden_dict, pop_samples, pheno_df)
    model <- '
      # direct effect
      comorbidity ~ c*gene

      # mediator
      bmi_residuals ~ a*gene
      comorbidity ~ b*bmi_residuals

      # indirect effect (a*b)
      ab := a*b

      # total effect
      total := c + (a*b)
    '

    fit <- sem(model, df, ordered = c("comorbidity"))
    summary_df = summary(fit)
    resdf = summary_df$pe
    resdf$gene <- ser$gene
    resdf$comorbidity <- ser$comorbidity
    resdf$nobs <- summary_df$data$nobs
    return(resdf)
}


In [13]:
# Load data
monogenic_meta_df <- read_csv("./monogenic_enrich_comorbid.csv")


Rows: 6 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): gene, gene_mask, comorbidity
dbl (4): OR, p_value, ci_low, ci_high

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [14]:
# Initialize an empty data frame to store results
results_df <- data.frame()

# Loop through each row of monogenic_meta_df
for (i in 1:nrow(monogenic_meta_df)) {
  # Extract the current row as a list
  ser <- monogenic_meta_df[i, ]
  cat(ser$gene)  
  
  # Call the run_sem_model function
  df <- run_sem_model(ser, gene_burden_dict, pop_samples, norm_pheno_df)
  
  # Append the results to the results_df
  results_df <- rbind(results_df, df)
}


BSNBSNGIGYF1SLTMBSNSLC5A3

In [15]:
results_df


,lhs,op,rhs,label,exo,est,se,z,pvalue,gene,comorbidity,nobs
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,comorbidity,~,gene,c,1,2.267921e-01,0.149271066,1.51933061,1.286793e-01,BSN,ht,454645
2,bmi_residuals,~,gene,a,1,5.809592e-01,0.110176669,5.27297846,1.342273e-07,BSN,ht,454645
3,comorbidity,~,bmi_residuals,b,0,2.997689e-01,0.001720804,174.20289116,0.000000e+00,BSN,ht,454645
4,comorbidity,|,t1,,0,4.625996e-01,0.001880730,245.96816959,0.000000e+00,BSN,ht,454645
5,comorbidity,~~,comorbidity,,0,9.106240e-01,0.000000000,NA,NA,BSN,ht,454645
6,bmi_residuals,~~,bmi_residuals,,0,9.945984e-01,0.002074297,479.48702857,0.000000e+00,BSN,ht,454645
7,gene,~~,gene,,1,1.341529e-04,0.000000000,NA,NA,BSN,ht,454645
10,bmi_residuals,~1,,,0,-7.795433e-05,0.001479223,-0.05269952,9.579713e-01,BSN,ht,454645
11,gene,~1,,,1,1.341706e-04,0.000000000,NA,NA,BSN,ht,454645


In [16]:
# Save results_df in CSV.gz format
write.csv(results_df, gzfile("sem_ukb.csv.gz"), row.names = FALSE)


In [17]:
# Upload results
system(paste0('dx upload sem_ukb.csv.gz --path /notebooks/bmi/data/downstream/sem/'), intern=T)

[1] "ID                                file-Gyqb82jJfzq8f4vpZ5kqyy05"     
 [2] "Class                             file"                              
 [3] "Project                           project-Gg0J78jJfzq8XV4v1G08XByZ"  
 [4] "Folder                            /notebooks/bmi/data/downstream/sem"
 [5] "Name                              sem_ukb.csv.gz"                    
 [6] "State                             closing"                           
 [7] "Visibility                        visible"                           
 [8] "Types                             -"                                 
 [9] "Properties                        -"                                 
[10] "Tags                              -"                                 
[11] "Outgoing links                    -"                                 
[12] "Created                           Thu Feb 20 16:10:51 2025"          
[13] "Created by                        dzb5732"                           
[14] " via the job                      job-GyqZyB8Jfzq7f3ypgx31qPKZ"      
[15] "Last modified                     Thu Feb 20 16:10:52 2025"          
[16] "Media type                        "                                  
[17] "archivalState                     \"live\""                          
[18] "cloudAccount                      \"cloudaccount-dnanexus\""